<a href="https://www.kaggle.com/code/dlu913/cyclistic-python?scriptVersionId=145904817" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import pandas as pd
import numpy as np
import seaborn as sns
import datetime 
import cufflinks as cf
from pandas.api.types import CategoricalDtype

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/opt/conda/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning:

The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.3-CAPI-1.16.1). Conversions between both will be slow.





# Ask

The Business:

Cyclistic is a fictional public bike sharing company allowing both member and non-members to ride bikes on a rental-based model.  The bike-share program has more than 5,800 bikes and 600 docking stations.The data comes from the company itself from users using the service. The data  falls within the ROCCC framework.  It is licensed by Motivate International LLC.

Lily Moreno is a director of marketing for Cyclistic.  As part of the capstone project,  we will be gathering, cleaning and analyzing data in order to provide insights for business decisions.




# Prepare

Import data from Kaggle. Data of each month is concated into one dataframe.

In [2]:
#Import CSV files

Jan_2021 = pd.read_csv('../input/bikeset/202101-divvy-tripdata.csv')
Feb_2021 = pd.read_csv('../input/bikeset/202102-divvy-tripdata.csv')
March_2021 = pd.read_csv('../input/bikeset/202103-divvy-tripdata.csv')
April_2021 = pd.read_csv('../input/bikeset/202104-divvy-tripdata.csv')
May_2021 = pd.read_csv('../input/bikeset/202105-divvy-tripdata.csv')
June_2021 = pd.read_csv('../input/bikeset/202106-divvy-tripdata.csv')
July_2021 = pd.read_csv('../input/bikeset/202107-divvy-tripdata.csv')
Aug_2021 = pd.read_csv('../input/bikeset/202108-divvy-tripdata.csv')
Sep_2021 = pd.read_csv('../input/bikeset/202109-divvy-tripdata.csv')
Oct_2021 = pd.read_csv('../input/bikeset/202110-divvy-tripdata.csv')
Nov_2021 = pd.read_csv('../input/bikeset/202111-divvy-tripdata.csv')
Dec_2021 = pd.read_csv('../input/bikeset/202112-divvy-tripdata.csv')

In [3]:
#Combine all CSV files into one dataset

bike_data = pd.concat([Jan_2021, Feb_2021, March_2021, April_2021, May_2021, June_2021, July_2021, Aug_2021, Sep_2021, Oct_2021, Nov_2021, Dec_2021], ignore_index = True)


bike_data






,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,E19E6F1B8D4C42ED,electric_bike,2021-01-23 16:14:19,2021-01-23 16:24:44,California Ave & Cortez St,17660,NaN,NaN,41.900341,-87.696743,41.890000,-87.720000,member
1,DC88F20C2C55F27F,electric_bike,2021-01-27 18:43:08,2021-01-27 18:47:12,California Ave & Cortez St,17660,NaN,NaN,41.900333,-87.696707,41.900000,-87.690000,member
2,EC45C94683FE3F27,electric_bike,2021-01-21 22:35:54,2021-01-21 22:37:14,California Ave & Cortez St,17660,NaN,NaN,41.900313,-87.696643,41.900000,-87.700000,member
3,4FA453A75AE377DB,electric_bike,2021-01-07 13:31:13,2021-01-07 13:42:55,California Ave & Cortez St,17660,NaN,NaN,41.900399,-87.696662,41.920000,-87.690000,member
4,BE5E8EB4E7263A0B,electric_bike,2021-01-23 02:24:02,2021-01-23 02:24:45,California Ave & Cortez St,17660,NaN,NaN,41.900326,-87.696697,41.900000,-87.700000,casual
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5595058,847431F3D5353AB7,electric_bike,2021-12-12 13:36:55,2021-12-12 13:56:08,Canal St & Madison St,13341,NaN,NaN,41.882289,-87.639752,41.890000,-87.610000,casual
5595059,CF407BBC3B9FAD63,electric_bike,2021-12-06 19:37:50,2021-12-06 19:44:51,Canal St & Madison St,13341,Kingsbury St & Kinzie St,KA1503000043,41.882123,-87.640053,41.889106,-87.638862,member
5595060,60BB69EBF5440E92,electric_bike,2021-12-02 08:57:04,2021-12-02 09:05:21,Canal St & Madison St,13341,Dearborn St & Monroe St,TA1305000006,41.881956,-87.639955,41.880254,-87.629603,member
5595061,C414F654A28635B8,electric_bike,2021-12-13 09:00:26,2021-12-13 09:14:39,Lawndale Ave & 16th St,362.0,NaN,NaN,41.860000,-87.720000,41.850000,-87.710000,member


In [4]:
bike_data.to_csv('mycsvfile.csv',index=False)



# Process

After the data has been concated,  the next step involves cleaning the data.  This is comprised of removing unecessary data.  Incorrect formatting, corrupt data and other errors will be resolved in this step.


In [5]:
bike_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5595063 entries, 0 to 5595062
Data columns (total 13 columns):
 #   Column              Dtype  
---  ------              -----  
 0   ride_id             object 
 1   rideable_type       object 
 2   started_at          object 
 3   ended_at            object 
 4   start_station_name  object 
 5   start_station_id    object 
 6   end_station_name    object 
 7   end_station_id      object 
 8   start_lat           float64
 9   start_lng           float64
 10  end_lat             float64
 11  end_lng             float64
 12  member_casual       object 
dtypes: float64(4), object(9)
memory usage: 554.9+ MB


In [6]:
 
bike_data.isnull().sum()

ride_id                    0
rideable_type              0
started_at                 0
ended_at                   0
start_station_name    690809
start_station_id      690806
end_station_name      739170
end_station_id        739170
start_lat                  0
start_lng                  0
end_lat                 4771
end_lng                 4771
member_casual              0
dtype: int64

In [7]:
#drop Null values
bike_data = bike_data.dropna(how='any',axis=0)

In [8]:
bike_data.isnull().sum()

ride_id               0
rideable_type         0
started_at            0
ended_at              0
start_station_name    0
start_station_id      0
end_station_name      0
end_station_id        0
start_lat             0
start_lng             0
end_lat               0
end_lng               0
member_casual         0
dtype: int64

In [9]:
#check for duplicates

bike_data.duplicated().any()

False

In [10]:
# changing the 'started_at' and 'ended_at' dtype to datetime
bike_data['started_at']= bike_data['started_at'].astype('datetime64')
bike_data['ended_at']= bike_data['ended_at'].astype('datetime64')

In [11]:
#Drop unnecessary columns

bike_data = bike_data.drop(columns=['start_station_name', 'start_station_id', 'end_station_name', 'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng'],axis='columns')

# Analyze

We begin to look at the data at a deeper level.  New columns are created in order to help us with this process.  

In [12]:

# Creating New Column "ride_length" and changing its datatype to "int32"
# In this column, each row contains the difference between "starting time" and "ending time" columns in minutes.

bike_data['ride_length'] = (bike_data['ended_at'] - bike_data['started_at'])/pd.Timedelta(minutes=1)
bike_data['ride_length'] = bike_data['ride_length'].astype('int32')


bike_data.head()

,ride_id,rideable_type,started_at,ended_at,member_casual,ride_length
9,B9F73448DFBE0D45,classic_bike,2021-01-24 19:15:38,2021-01-24 19:22:51,member,7
10,457C7F4B5D3DA135,electric_bike,2021-01-23 12:57:38,2021-01-23 13:02:10,member,4
11,57C750326F9FDABE,electric_bike,2021-01-09 15:28:04,2021-01-09 15:37:51,casual,9
12,4D518C65E338D070,electric_bike,2021-01-09 15:28:57,2021-01-09 15:37:54,casual,8
13,9D08A3AFF410474D,classic_bike,2021-01-24 15:56:59,2021-01-24 16:07:08,casual,10


In [13]:
# Sorting Values by "ride_length" column in Ascending order.

bike_data.sort_values(by = 'ride_length')



,ride_id,rideable_type,started_at,ended_at,member_casual,ride_length
5129636,FD8AF7324ABAE9DA,electric_bike,2021-11-07 01:56:51,2021-11-07 01:00:57,casual,-55
5113400,508B09A5FB0737DC,classic_bike,2021-11-07 01:54:50,2021-11-07 01:00:45,member,-54
5270218,7AECC76D1562B51C,classic_bike,2021-11-07 01:54:58,2021-11-07 01:01:29,casual,-53
5248365,6F9E76F5EDAAC1B8,electric_bike,2021-11-07 01:55:42,2021-11-07 01:01:55,member,-53
5091563,CDB307B8494885AD,electric_bike,2021-11-07 01:55:09,2021-11-07 01:02:26,casual,-52
...,...,...,...,...,...,...
2613756,99415B003B7E7EEE,docked_bike,2021-07-08 19:29:49,2021-08-11 21:56:58,casual,49107
1325528,E6E1E5BFFD72ECEC,docked_bike,2021-06-05 23:33:51,2021-07-12 13:55:14,casual,52701
758503,BDA1217EC8532C7B,docked_bike,2021-05-02 02:56:07,2021-06-08 13:37:43,casual,53921
1789291,7F0578ABF030FC83,docked_bike,2021-06-04 22:03:33,2021-07-13 14:15:14,casual,55691


In [14]:
# Some months had too many negative values
# Such errors happened because the "ending time" is earlier than the "starting time" in their respective rows.

bike_data[bike_data['ride_length'] < 0].count()

# Number of rows containing Negative Values.

ride_id          51
rideable_type    51
started_at       51
ended_at         51
member_casual    51
ride_length      51
dtype: int64

In [15]:
bike_data[bike_data['ride_length'] < 1].count()

# Number of rows containing "ride length" less than "1" minute.


ride_id          59369
rideable_type    59369
started_at       59369
ended_at         59369
member_casual    59369
ride_length      59369
dtype: int64

In [16]:
bike_data = bike_data[bike_data['ride_length'] >= 1]
bike_data = bike_data.reset_index()
bike_data = bike_data.drop(columns=['index'])

# Removing 80845 rows containing negative values & ride length less than 1 minute. 
# Any trips that were below 60 seconds in length are potentially false starts or users trying to re-dock a bike to ensure it was secure.


In [17]:
bike_data = bike_data.astype({'ride_id':'string', 'rideable_type':'category', 'member_casual':'category'})

# Coverting datatypes of each columns.

bike_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4528933 entries, 0 to 4528932
Data columns (total 6 columns):
 #   Column         Dtype         
---  ------         -----         
 0   ride_id        string        
 1   rideable_type  category      
 2   started_at     datetime64[ns]
 3   ended_at       datetime64[ns]
 4   member_casual  category      
 5   ride_length    int32         
dtypes: category(2), datetime64[ns](2), int32(1), string(1)
memory usage: 129.6 MB


In [18]:
# Checking if Dataframe "df" contains any missing values.

bike_data.isna().sum()

ride_id          0
rideable_type    0
started_at       0
ended_at         0
member_casual    0
ride_length      0
dtype: int64

# Share

In this step, graphs are produced using plotly.  We can begin to share our insights by visualizing the data.  By doing this, stakeholders have a much more concise and clear view of the data.  This is a much more efficient way of interpreting information, rather than just looking at a huge dataframe.

In [19]:
# Creating new columns "year", "month", "day_of_week", "hour" and Converting datatypes.


bike_data['year'] = bike_data['started_at'].dt.year

cats1 = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
bike_data['month'] = bike_data['started_at'].dt.month_name()
bike_data['month'] = bike_data['month'].astype(CategoricalDtype(categories=cats1, ordered=False))

cats2 = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
bike_data['day_of_week'] = bike_data['started_at'].dt.day_name()
bike_data['day_of_week'] = bike_data['day_of_week'].astype(CategoricalDtype(categories=cats2, ordered=False))

bike_data['hour'] = bike_data['started_at'].dt.hour

bike_data = bike_data.astype({'year':'int16', 'hour':'int8'})


bike_data.head()

,ride_id,rideable_type,started_at,ended_at,member_casual,ride_length,year,month,day_of_week,hour
0,B9F73448DFBE0D45,classic_bike,2021-01-24 19:15:38,2021-01-24 19:22:51,member,7,2021,January,Sunday,19
1,457C7F4B5D3DA135,electric_bike,2021-01-23 12:57:38,2021-01-23 13:02:10,member,4,2021,January,Saturday,12
2,57C750326F9FDABE,electric_bike,2021-01-09 15:28:04,2021-01-09 15:37:51,casual,9,2021,January,Saturday,15
3,4D518C65E338D070,electric_bike,2021-01-09 15:28:57,2021-01-09 15:37:54,casual,8,2021,January,Saturday,15
4,9D08A3AFF410474D,classic_bike,2021-01-24 15:56:59,2021-01-24 16:07:08,casual,10,2021,January,Sunday,15


In [20]:
example_1 = bike_data.groupby('member_casual', as_index=False).count()


import plotly.express as px

px.bar(example_1, y = 'member_casual', x = 'ride_id', range_x = [0,3000000],
        color = 'member_casual', 
        height = 300,
        text = 'ride_id', 
        labels = {'ride_id': 'No. of Rides', 'member_casual': 'Member/Casual'},
        hover_name = 'member_casual', hover_data = {'member_casual': False, 'month': False, 'ride_id': True}, 
        color_discrete_map = {'casual': '#FF934F', 'member': '#058ED9'})

In [21]:
#create pivot table 

bike_data_pv1 = pd.pivot_table(bike_data,
              index = ['year', 'month', 'member_casual'],
              values = 'ride_id',
              aggfunc = ['count'],
              margins = True,
              margins_name = 'Total Count')
bike_data_pv1 = bike_data_pv1.loc[(bike_data_pv1 != 0).any(axis=1)]
bike_data_pv1

count
                                     ride_id
year        month     member_casual         
2021        January   casual           14583
                      member           68039
            February  casual            8508
                      member           33793
            March     casual           75059
                      member          128349
            April     casual          119362
                      member          175261
            May       casual          214652
                      member          230505
            June      casual          300769
                      member          299743
            July      casual          365513
                      member          317690
            August    casual          338138
                      member          327946
            September casual          290078
                      member          323287
            October   casual          187324
                      member          284140
            November  casual           69267
                      member          182922
            December  casual           44684
                      member          129321
Total Count                          4528933

In [22]:
example_2 = bike_data.groupby(['year', 'month', 'member_casual'], as_index=False).count()

example_2 = example_2[example_2['ride_id'] != 0]

px.line(example_2, title = 'Riders Per Month', x = 'month', y = 'ride_id', range_y = [0,450000],
        color = 'member_casual', 
        line_shape = 'spline',
        markers=True, 
        labels = {'ride_id': 'No. of Rides', 'month': 'Months (Jan 2021 - Dec 2021)', 'member_casual': 'Member/Casual'},
        hover_name = 'member_casual', hover_data = {'member_casual': False, 'month': True, 'ride_id': True}, 
        color_discrete_map = {'casual': '#FF934F', 'member': '#058ED9'})

<h1>Average Number of Rides During Weekday</h1>

In [23]:
pd.pivot_table(bike_data,
              index = ['day_of_week', 'member_casual'],
              values = 'ride_id',
              aggfunc = ['count'],
              margins = True,
              margins_name = 'Total Count')

count
                           ride_id
day_of_week member_casual         
Sunday      casual          399565
            member          305877
Monday      casual          226616
            member          341154
Tuesday     casual          212811
            member          382509
Wednesday   casual          215996
            member          392003
Thursday    casual          222057
            member          368074
Friday      casual          287202
            member          360223
Saturday    casual          463690
            member          351156
Total Count                4528933

In [24]:
example_3 = bike_data.groupby(['day_of_week', 'member_casual'], as_index=False).count()

px.line(example_3, title= 'Average number of Riders per Weekday', x = 'day_of_week', y = 'ride_id', range_y = [0,600000],
        color = 'member_casual',
        line_shape = 'spline',
        markers=True,
        labels = {'ride_id': 'No. of Rides', 'day_of_week': 'Weekdays', 'member_casual': 'Member/Casual'},
        hover_name = 'member_casual', hover_data = {'member_casual': False, 'month': False, 'ride_id': True}, 
        color_discrete_map = {'casual': '#FF934F', 'member': '#058ED9'})



In [25]:
# Bar graph of Average number of riders per weekday

example_3a = bike_data.groupby(['day_of_week', 'member_casual'], as_index=False).count()


example_3a = px.histogram(example_3a, title= 'Average number of Riders per Weekday', x = 'day_of_week', y = 'ride_id', range_y = [0,600000],
        color = 'member_casual',
        barmode='group',
        labels = {'ride_id': 'No. of Rides', 'day_of_week': 'Weekdays', 'member_casual': 'Member/Casual'},
        color_discrete_map = {'casual': '#FF934F', 'member': '#058ED9'})

example_3a.show()





<h1>Average Number of Rides per hour</h1>

In [26]:
pd.pivot_table(bike_data,
              values = 'ride_id',
              index = ['hour', 'member_casual'],
              aggfunc = ['count'],
              margins = True,
              margins_name = 'Total Count')

count
                           ride_id
hour        member_casual         
0           casual           41892
            member           24846
1           casual           30386
            member           16072
2           casual           19404
            member            8818
3           casual           10132
            member            5011
4           casual            6627
            member            5903
5           casual            8729
            member           25263
6           casual           19232
            member           70710
7           casual           35730
            member          128697
8           casual           49257
            member          147593
9           casual           60278
            member          106619
10          casual           84477
            member          103803
11          casual          111235
            member          125008
12          casual          131981
            member          144697
13          casual          140799
            member          141855
14          casual          144689
            member          139677
15          casual          151781
            member          162454
16          casual          166238
            member          213362
17          casual          194047
            member          269474
18          casual          174502
            member          224951
19          casual          133682
            member          158682
20          casual           96994
            member          104581
21          casual           81950
            member           76014
22          casual           75698
            member           57638
23          casual           58197
            member           39268
Total Count                4528933

In [27]:
example_4 = bike_data.groupby(['hour', 'member_casual'], as_index=False).count()

example_4 = px.line(example_4, title = 'Number of Riders per Hour', x = 'hour', y = 'ride_id', range_x = [0,23], range_y = [0,350000],
                 color = 'member_casual',
                 line_shape = 'spline',
                 markers=True,
                 labels = {'ride_id': 'No. of Rides', 'hour': '24 Hours', 'member_casual': 'Member/Casual'},
                 hover_name = 'member_casual', hover_data = {'member_casual': False, 'month': False, 'ride_id': True},
                 color_discrete_map = {'casual': '#FF934F', 'member': '#058ED9'})

example_4.update_xaxes(dtick=1)



In [28]:
#Average Ride Length Per Year

pd.pivot_table(bike_data,
               index = 'member_casual',
               values = ['ride_length'],
               aggfunc = ['mean'],
               margins = True,
               margins_name = 'Total Ride Average')



,mean
,ride_length
member_casual,
casual,32.341510
member,12.891249
Total Ride Average,21.600564


In [29]:
#Calculate Mode of ride length between members and casual riders.
bike_data.groupby(['member_casual'])['ride_length'].agg(pd.Series.mode)

member_casual
casual    8
member    5
Name: ride_length, dtype: int32

<h1>Average Ride Length</h1>

In [30]:
example_5 = round(bike_data.groupby('member_casual', as_index=False).mean(),2)

px.bar(example_5, y = 'member_casual', x = 'ride_length', range_x = [0,35],
        color = 'member_casual', 
        height = 300,
        text = 'ride_length', 
        labels = {'ride_length': 'Average Ride Length (minutes)', 'member_casual': 'Member/Casual'},
        hover_name = 'member_casual', hover_data = {'member_casual': False, 'ride_length': True}, 
        color_discrete_map = {'casual': '#FF934F', 'member': '#058ED9'})

In [31]:
#Average Ride Length Per month

pd.pivot_table(bike_data,
               index = ['year', 'month', 'member_casual'],
               values = ['ride_length'],
               aggfunc = ['mean'],
               margins = True,
               margins_name = 'Total Ride Average')

mean
                                           ride_length
year               month     member_casual            
2021               January   casual          26.069190
                             member          11.673099
                   February  casual          47.222379
                             member          14.540141
                   March     casual          38.284683
                             member          13.349539
                   April     casual          38.287110
                             member          13.971425
                   May       casual          39.541556
                             member          14.073933
                   June      casual          38.461933
                             member          13.870062
                   July      casual          33.143634
                             member          13.519651
                   August    casual          28.350650
                             member          13.261491
                   September casual          27.868973
                             member          12.833021
                   October   casual          26.070541
                             member          11.716369
                   November  casual          22.213262
                             member          10.631936
                   December  casual          24.568548
                             member          10.245853
Total Ride Average                           21.600564

In [32]:
example_6 = round(bike_data.groupby(['year', 'month', 'member_casual'], as_index=False).mean(),2).dropna()

px.bar(example_6,title = 'Average Ride Length per Month', x = 'month', y = 'ride_length',
        color = 'member_casual',
        barmode='group',
        text = 'ride_length', 
        labels = {'ride_length': 'Average Ride Length (minutes)', 'member_casual': 'Member/Casual', 'month': 'Months (Oct 2020 - Sep 2021)'},
        hover_name = 'member_casual', hover_data = {'member_casual': False, 'ride_length': True}, 
        color_discrete_map = {'casual': '#FF934F', 'member': '#058ED9'})

In [33]:
#Average Ride Length per Weekday

pd.pivot_table(bike_data,
               index = ['day_of_week', 'member_casual'],
               values = ['ride_length'],
               aggfunc = ['mean'],
               margins = True,
               margins_name = 'Total Ride Average')

mean
                                 ride_length
day_of_week        member_casual            
Sunday             casual          37.502679
                   member          14.960991
Monday             casual          32.475853
                   member          12.426376
Tuesday            casual          28.595185
                   member          12.071878
Wednesday          casual          28.053232
                   member          12.143032
Thursday           casual          27.786001
                   member          12.038180
Friday             casual          30.728808
                   member          12.492234
Saturday           casual          34.725853
                   member          14.571279
Total Ride Average                 21.600564

In [34]:
example_7 = round(bike_data.groupby(['day_of_week', 'member_casual'], as_index=False).mean(),2)

px.bar(example_7, x = 'day_of_week', y = 'ride_length',
        color = 'member_casual',
        barmode='group',
        text = 'ride_length', 
        labels = {'ride_length': 'Average Ride Length (minutes)', 'member_casual': 'Member/Casual', 'day_of_week': 'Weekdays'},
        hover_name = 'member_casual', hover_data = {'member_casual': False, 'ride_length': True}, 
        color_discrete_map = {'casual': '#FF934F', 'member': '#058ED9'})

# Act

Members have a steady rate of riders during the weekday when people have to get to work.  Casual Riders spike on the weekend. There is a spike of both casual and member riders during the morning (around 9pm) and evening hours (around 6pm) when people are commuting to and off work.  However,  there are still more member riders during these times.

**Recommendations**

As a business strategy,  doubling down on casual riders on the weekend would create a great source of continued revenue.  Increasing marketing campaigns and incentives to get more people to ride on weeekends would be a viable move for  attracting more riders. 

Scaling more riders in with multiple ridership packages can be an attractive model.  A second order effect of converting them to members would be a good pivot (incentivized cheaper rates for members).

A gamification system of giving riders perks based on rider usage could be a new alluring model.  Rewards such as free miles can be an attractive perk.  Members can also spare or give miles to other family members or friends to use.



In [35]:
bike_data.to_csv('mycsvfile.csv',index=False)